In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
df_qud = pd.read_csv('/content/drive/MyDrive/Planification Projet Capstone/QUD.csv', header=None)
df_qud.columns = ['Occupancy', 'Power Consumption', 'Normalized Power Consumption', 'P(t)-P(t+1)', 'P(t)-P(t-1)', 'Micro-Moment Class']
df_qud

,Occupancy,Power Consumption,Normalized Power Consumption,P(t)-P(t+1),P(t)-P(t-1),Micro-Moment Class
0,1,0.00,-0.002259,0.00,0.00,0
1,1,0.00,-0.002259,-166.43,0.00,1
2,1,166.43,0.007632,0.00,166.43,3
3,1,159.55,0.007223,0.00,0.00,3
4,1,143.27,0.006255,0.00,0.00,3
...,...,...,...,...,...,...
46925,1,-1.64,-0.002357,0.00,0.00,0
46926,1,-10.64,-0.002891,0.00,0.00,0
46927,1,79.22,0.002449,0.00,0.00,0
46928,1,74.38,0.002161,0.00,0.00,0


In [3]:
df_dred = pd.read_csv('/content/drive/MyDrive/Planification Projet Capstone/DRED.csv', header=None)
df_dred.columns = ['Occupancy', 'Power Consumption', 'Normalized Power Consumption', 'P(t)-P(t+1)', 'P(t)-P(t-1)', 'Micro-Moment Class']
df_dred

,Occupancy,Power Consumption,Normalized Power Consumption,P(t)-P(t+1),P(t)-P(t-1),Micro-Moment Class
0,1,0.0,-0.68907,0.0,0.0,0
1,1,0.0,-0.68907,0.0,0.0,0
2,0,0.0,-0.68907,0.0,0.0,0
3,1,0.0,-0.68907,0.0,0.0,0
4,0,0.0,-0.68907,0.0,0.0,0
...,...,...,...,...,...,...
166348,1,0.0,-0.68907,0.0,0.0,0
166349,0,0.0,-0.68907,0.0,0.0,0
166350,0,0.0,-0.68907,0.0,0.0,0
166351,1,0.0,-0.68907,0.0,0.0,0


In [4]:
# Rajouter les colonnes timestep et compte
n_rows = df_dred.shape[0]
#df_dred['Timestep'] = [2]*n_rows
#df_dred['Count'] = [i for i in range(n_rows)]
df_dred.drop('Normalized Power Consumption', axis=1, inplace=True)
#df_qud.drop('Normalized Power Consumption', axis=1, inplace=True)

In [ ]:
def sliding_window(dataset, timestep):
    
    dataset = dataset.to_numpy().astype(np.float32)
    timesteps = []
    
    for i in range(dataset.shape[0]):
        if i < dataset.shape[0] - timestep:
            timesteps.append(dataset[i:i+timestep])

    return timesteps

In [ ]:
#qud_timesteps = sliding_window(df_qud, 3)
#dred_timesteps = sliding_window(df_dred, 3)
#dred_timesteps[50]

In [ ]:
'''
# TODO: Cette fonction est plutôt lente et pourrait probablement être optimisée
def calculate_covariance_from_timewindow(timesteps, col):

    new_timesteps = np.ones((len(timesteps), timesteps[0].shape[0], timesteps[0].shape[1]+1))

    for i, window in enumerate(timesteps):
        # Calcul de la covariance de la fenêtre
        mean = np.mean(window[:, col])
        max = np.max(window[:, col])
        min = np.min(window[:, col])
        std = np.std(window[:, col])

        if mean > 0:
            covariance = ((max-min) / 100) * (std / mean) 
        else:
            covariance = 0

        # Création de la nouvelle fenêtre temporelle avec la covariance
        new_window = np.ones((timesteps[0].shape[0], timesteps[0].shape[1]+1))

        for j, timestep in enumerate(window):
            new_timestep = new_window[j] * np.append(timestep, [1], axis=0)
            new_timestep[-1] = new_timestep[-1] * covariance
            new_window[j] = new_window[j] * new_timestep

        new_timesteps[i] = new_timesteps[i] * new_window
    
    return new_timesteps'''

'\n# TODO: Cette fonction est plutôt lente et pourrait probablement être optimisée\ndef calculate_covariance_from_timewindow(timesteps, col):\n\n    new_timesteps = np.ones((len(timesteps), timesteps[0].shape[0], timesteps[0].shape[1]+1))\n\n    for i, window in enumerate(timesteps):\n        # Calcul de la covariance de la fenêtre\n        mean = np.mean(window[:, col])\n        max = np.max(window[:, col])\n        min = np.min(window[:, col])\n        std = np.std(window[:, col])\n\n        if mean > 0:\n            covariance = ((max-min) / 100) * (std / mean) \n        else:\n            covariance = 0\n\n        # Création de la nouvelle fenêtre temporelle avec la covariance\n        new_window = np.ones((timesteps[0].shape[0], timesteps[0].shape[1]+1))\n\n        for j, timestep in enumerate(window):\n            new_timestep = new_window[j] * np.append(timestep, [1], axis=0)\n            new_timestep[-1] = new_timestep[-1] * covariance\n            new_window[j] = new_window[j]

In [ ]:
#new_qud = calculate_covariance_from_timewindow(qud_timesteps, [1])
'''new_dred = calculate_covariance_from_timewindow(dred_timesteps, 1)
new_dred = calculate_covariance_from_timewindow(new_dred, 2)
new_dred = calculate_covariance_from_timewindow(new_dred, 3)'''

'new_dred = calculate_covariance_from_timewindow(dred_timesteps, 1)\nnew_dred = calculate_covariance_from_timewindow(new_dred, 2)\nnew_dred = calculate_covariance_from_timewindow(new_dred, 3)'

In [5]:
class StatefulScaler():
    def __init__(self, epsilon, initial_mean = 0): # e.g. epsilons: {1: 0.1, 2: 0.9}
        self.epsilon = epsilon
        self.stateful_mean = initial_mean
        self.data_mean = initial_mean

    def normalize(self, fenetre, update_mean = True):

        if update_mean:
            self.stateful_mean = (1-self.epsilon) * self.stateful_mean + (self.data_mean * self.epsilon)
        
        factor = (self.stateful_mean-self.data_mean) / (self.data_mean + 1e-7)
        fenetre_norm = (fenetre - fenetre.min()) / ((fenetre.max() - fenetre.min()) + 1e-7) # Ajouter 1e-7 a la moyenne pour éviter la division par 0
        fenetre_norm = fenetre_norm * factor
        
        return fenetre_norm

    def normalize_window(self, window):
        fenetre_norm = np.asarray([self.normalize(window[i]) for i in range(window.shape[0])]).astype(np.float32)
        return fenetre_norm

    def reset_stateful_mean(self):
        self.stateful_mean = self.data_mean


In [6]:
class Generateur(tf.keras.utils.Sequence):

    def __init__(self, dataset, batch_size=8, window_size=3):

        self.X , self.y = dataset.drop('Micro-Moment Class', axis=1).to_numpy().astype(np.float32), np.asarray(tf.one_hot(dataset['Micro-Moment Class'], depth=5)).astype(np.float32)
        
        self.batch_size = batch_size
        self.window_size = window_size

        self.t_scaler = StatefulScaler(0.75, np.mean(self.X[:, 1]))
        self.t_fwd_scaler = StatefulScaler(0.9, np.mean(abs(self.X[:, 2])))
        self.t_back_scaler = StatefulScaler(0.9, np.mean(abs(self.X[:, 3])))
        
    def on_epoch_end(self):
        # Reset Stateful mean to dataset mean
        self.t_scaler.reset_stateful_mean()
        self.t_fwd_scaler.reset_stateful_mean()
        self.t_back_scaler.reset_stateful_mean()
        
    def __len__(self):
        return (self.X.shape[0] - self.window_size + 1) // self.batch_size

    def __getitem__(self, idx):

        batch_x = self.X[ (idx * self.batch_size) : (idx + 1) * self.batch_size + self.window_size - 1, : ]
        batch_y = self.y[ (idx * self.batch_size) : (idx + 1) * self.batch_size + self.window_size - 1, : ]

        batch_x, batch_y = self.slide_window([batch_x, batch_y])
        
        # Normalisation de P(t), P(t) - P(t-1) et P(t) - P(t+1)
        for i in range(self.batch_size):
            batch_x[:, :, 1] = self.t_scaler.normalize(batch_x[i, :, 1])
            batch_x[:, :, 2] = self.t_fwd_scaler.normalize(batch_x[i, :, 2])
            batch_x[:, :, 3] = self.t_back_scaler.normalize(batch_x[i, :, 3])

        return np.asarray(batch_x).astype(np.float32) , np.asarray(batch_y).astype(np.float32)

    def slide_window(self, batch, timestep=3):

        batch_x, batch_y = batch

        X, y = [], []
        
        for i in range(batch_x.shape[0]):
            if i <= batch_x.shape[0] - timestep:
                y.append(batch_y[i+(timestep // 2) , :])
                X.append(batch_x[i:i+timestep, :])

        return np.asarray(X).astype(np.float32), np.asarray(y).astype(np.float32)

In [14]:
# Modele
input = tf.keras.Input(shape=(3, 4))

output = tf.keras.layers.Dense(128, activation='relu')(input)
output = tf.keras.layers.LSTM(256)(output)
output = tf.keras.layers.Dense(128, activation='relu')(output)

output = tf.keras.layers.Dense(5, activation='softmax')(output)

model = tf.keras.Model(inputs=input, outputs = output)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3, 4)]            0         
                                                                 
 dense_3 (Dense)             (None, 3, 128)            640       
                                                                 
 lstm_1 (LSTM)               (None, 256)               394240    
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dense_5 (Dense)             (None, 5)                 645       
                                                                 
Total params: 428,421
Trainable params: 428,421
Non-trainable params: 0
_________________________________________________________________


In [15]:
#df_dred = df_dred[:1000]

dred_train = df_dred[:int(df_dred.shape[0]*0.8)]
dred_test = df_dred[int(df_dred.shape[0]*0.8):]

print(dred_train.head())

   Occupancy  Power Consumption  P(t)-P(t+1)  P(t)-P(t-1)  Micro-Moment Class
0          1                0.0          0.0          0.0                   0
1          1                0.0          0.0          0.0                   0
2          0                0.0          0.0          0.0                   0
3          1                0.0          0.0          0.0                   0
4          0                0.0          0.0          0.0                   0


In [ ]:
df_qud = df_qud[:1000]
qud_train = df_qud[:int(df_qud.shape[0]*0.8)].reset_index()
qud_test = df_qud[int(df_qud.shape[0]*0.8):].reset_index()

In [16]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = "categorical_crossentropy",
    metrics = [get_f1, 'accuracy']
)

In [17]:
# DRED

generateur_train = Generateur(dred_train, window_size=3, batch_size=256)
generateur_test = Generateur(dred_train, window_size=3, batch_size=256)

print(generateur_train[0])
dred_hist = model.fit(
    x = generateur_train,
    epochs=10,
    validation_data = generateur_test
)

(array([[[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.]]], dtype=float32), array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32))
Epoch 1/10
519/519 [==============================] - 86s 56ms/step - loss: 0.6304 - get_f1: 0.6950 - accuracy: 0.7357 - val_loss: 0.5434 - val_get_f1: 0.7714 - val_accuracy: 0.7714
Epoch 2/10
519/519 [==============================] - 27s 53ms/step - loss: 0.5458 - get_f1: 0.7563 - acc

In [18]:
tf.keras.models.save_model(model, '/content/drive/MyDrive/Planification Projet Capstone/baseline_model.h5')

In [ ]:
# QUD
qud_hist = model.fit(
    x = Generateur(qud_train, window_size=3),
    epochs=100,
    validation_data = Generateur(qud_test, window_size=3)
)

In [10]:
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
np.max(hist.history['val_get_f1'])

0.9079861044883728

In [ ]:
np.max(qud_hist.history['val_get_f1'])

0.5801874995231628